<a href="https://colab.research.google.com/github/nathasya-abenita/MA2151-Simulation-and-Mathematical-Computation./blob/main/Pit%20Viper%20and%20Rodent%20Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Akhir
MA2151 Simulasi dan Komputasi Matematika \\
oleh \\
Kelompok 15
* Tharisa Alviolita	           (10120044)
* Nathasya Abenita Christien   (10120080)
* Mohammad Arbi Wijaya         (10120096)
* Chantika Constantine         (10120098) 
* Jevan Christopher Aryento    (10820010) 

# Program Kasus Ke-1 (Rodent bergerak normal dengan acak)
Pada program ini, dihasilkan matriks-matriks lingkungan dari simulasi untuk tiap iterasi yang disimpan pada matriks *matDisplay*. \\
Berikut beberapa informasi pemaknaan tiap variabel yang ada.
*   mat[i, j, 0] = matStatus
*   mat[i, j, 1] = matTemperature 
*   Kode status: -1 (border), 0 (empty), 1 (viper), 2 (rodent)
*   Info agen viper: agents[0,0] (row), agents[0,1] (column)
*   Info agen rodent: agents[1,0] (row), agents[1,1] (column), agents[1,2] (temperature)

Berikut asumsi-asumsi yang digunakan.
*   Absorbing border condition
*   Ketetanggaan Moore
* Ukuran matriks lingkungan 25x25
* Konstanta difusi 0.1



In [ ]:
# PROGRAM ULAR (VIPER) MEMANGSA SEEKOR HEWAN PENGERAT (RODENT)
# Program ini akan memodelkan gerakan ular memangsa seekor pengerat dengan nilai-nilai inisialisasi tertentu, yaitu banyak
# baris-kolom lingkungan, konstanta difusi suhu, dan suhu lingkungan.
# Input (Nilai inisialisasi): m (baris), n (kolom), konstanta difusi suhu, suhu lingkungan
# Output: Matriks-matriks lingkungan pada tiap iterasi yang memodelkan posisi viper dan rodent yang nantinya akan dianimasikan
import numpy as np

# Fungsi untuk algoritma penginisialisasi lingkungan
def InitializeEnvironment (m, n, ENV_TEMP):
  # Matriks lingkungan utama diinisialisasi terlebih dahulu dengan nilai 0 (kode status empty)
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] adalah matStatus, sedangkan mat[i,j,1] adalah matTemperature
  # Mengubah status pada bagian batas ke status border (-1) dan meng-assign suhu lingkungan ke semua sel
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1: # Untuk sel border
        mat[i][j][0] = -1 # Status diubah ke kode border
        mat[i][j][1] = ENV_TEMP # Suhu batas adalah suhu lingkungan sehingga berlaku absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP # Untuk sel-sel di dalam, cukup di-assign suhu ke suhu lingkungan
        # dan status tetap sama, yaitu empty cells berkode 0
  # Inisialisasi kedua agen, yaitu viper dan rodent
  agents = []
  # Agen viper di-assign informasi posisi random pada lingkungan (baris, kolom)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Agen rodent di-assign informasi posisi random di lingkungan dan suhu acak dengan rata-rata 37 derajat celcius (baris, kolom, suhu)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  # Mengatur posisi kedua agen dan suhu rodent ke matriks lingkungan
  mat[agents[0][0]][agents[0][1]][0] = 1 # kode status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # kode status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # suhu pada posisi rodent disesuaikan ke suhu rodent
  return mat, agents

# Fungsi untuk algoritma peng-update status matriks lingkungan sesuai pergerakan agen
def UpdateStatus (mat, agents):
  # Akan dipanggil fungsi pergerakan rodent terlebih dahulu kemudian pergerakan viper
  mat, agents = RodentWalk(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

# Fungsi untuk algoritma pergerakan rodent
def RodentWalk (mat, agents):
  # Pemerolehan informasi posisi baris dan kolom serta suhu rodent pada matriks lingkungan
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  # Inisialisasi array penyimpan informasi array posisi tetangga rodent yang kosong
  movementChoice = []
  # Akan dicek kosong tidaknya kedelapan tetangga (ketetanggaan Moore) dari posisi rodent
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # jika status pada sel tersebut kosong
      movementChoice.append([row+x, column+y])
  # Ditambahkan juga kemungkinan rodent diam dengan posisi baris-kolom yang sama dengan awaln
  movementChoice.append([row, column]) 
  # Akan dipilih secara random posisi baru memungkinkan dan disimpan pada variabel newPosition
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  # Akan di-assign posisi baru rodent ke matriks lingkungan
  if newPosition[0] == row and newPosition[1] == column: # Jika posisi sama dengan sebelumnya,
    pass # tidak dilakukan perubahan
  else: # Jika posisinya baru,
    newRow, newColumn = newPosition[0], newPosition[1]
    # Matriks lingkungan di-update sesuai posisi baris-kolom baru
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Informasi agen rodent juga di-update sesuai posisi baru serta temperatur acak baru
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

# Fungsi untuk algoritma pegerakan viper
def ViperWalk (mat, agents):
  # Pemerolehan informasi posisi viper pada matriks lingkungan
  row, column = agents[0][0], agents[0][1]
  # Inisialisasi array penyimpan sel tetangga kosong atau ada rodent, suhunya, dan 
  # akhirnya pilihan-pilihan sel tetangga yang paling tepat
  emptyNeighbors, emptyNeighborsT, movementChoice = [], [], []
  # Akan dikumpulkan tetangga yang kosong atau terdapat rodent
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # jika status berkode 0 atau 2 (kosong atau ada rodent)
      emptyNeighbors.append([row+x, column+y])
  # Akan dikumpulkan suhu-suhu dari sel tetangga yang telah dikumpulkan
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  # Dicari suhu maksimumnya
  maxT = max(emptyNeighborsT)
  # Sel tetangga dengan suhu maksimum akan menjadi pilihan sel tetangga yang tepat
  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  # Akan dipilih secara acak pilihan yang ada sebagai posisi baru viper
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  # Matriks lingkungan di-update sesuai posisi baru viper
  newRow, newColumn = newPosition[0], newPosition[1]
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Informasi agen viper juga di-update sesuai posisi barunya
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

# Fungsi untuk algoritma difusi suhu matriks lingkungan
def HeatDiffusion (m, n, mat, DIFF_RATE):
  # Akan dilakukan difusi suhu pada tiap sel selain border
  for i in range (1, m+1):
    for j in range (1, n+1):
      # Suhu jumlah ke-8 tetangga suatu sel akan dihitung
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      # Suhu baru suatu sel akan mengikuti rumus berikut
      mat[i][j][1] = (1 - 8*DIFF_RATE)* mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

# Fungsi untuk algoritma pengecek sudah tidaknya mangsa ditangkap viper
def CheckPrey(viper, rodent):
  # Akan dicek apakah posisi viper dan rodent sama
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr: # Jika sama,
    return True # dikembalikan nilai benar
  else: # Jika beda, dikembalikan nilai salah
    return False

# Algoritma Utama
m, n, ENV_TEMP, DIFF_RATE = 25, 25, 30, 0.1 # Inisialisasi empat nilai yang diperlukan
matDisplay = [] # Inisialisasi array penyimpan matriks lingkungan pada tiap iterasi
# Algoritma dijalankan dengan proses looping dari fase 0-1-2 kemudian kembali ke 1-2 dan seterusnya
# hingga rodent dimangsa (viper dan rodent pada posisi yang sama) yang akan memberhentikan simulasi
phase = 0
while phase != 3:
  if phase == 0:
    mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
    phase += 1
  elif phase == 1:
    mat = HeatDiffusion(m, n, mat, DIFF_RATE)
    phase += 1
    # Pada tahap ini, matriks lingkungan pada suatu iterasi sudah lengkap dan dapat disimpan
    matDisplay.append(mat)
  elif phase == 2:
    mat, agents = UpdateStatus(mat, agents)
    print(agents)
    # Mengecek benar tidaknya rodent telah dimangsa
    if CheckPrey(agents[0], agents[1]):
      phase += 1
    else:
      phase = 1
# Setelah looping selesai, animasi simulasi dapat dibuat berdasarkan matDisplay, penyimpan semua matriks lingkungan pada tiap iterasi

[[2, 11], [7, 16, 36.82296726203919]]
[[3, 11], [7, 15, 36.683789326087535]]
[[2, 12], [7, 15, 36.683789326087535]]
[[1, 12], [6, 16, 37.08984478466989]]
[[2, 11], [7, 17, 36.91887064125419]]
[[3, 12], [6, 17, 37.024730095430876]]
[[4, 13], [5, 17, 37.17352975908292]]
[[5, 14], [5, 16, 37.32288332876408]]
[[5, 15], [4, 17, 36.85806872209132]]
[[4, 16], [4, 17, 36.85806872209132]]
[[3, 16], [3, 16, 37.29495674163117]]


# Animasi Simulasi Kasus Ke-1
Program sebelumnya telah ditambahkan dengan tambahan sintaks untuk membuat animasi dari matriks-matriks lingkungan untuk tiap iterasi yang tersimpan pada *matDisplay*

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation


def InitializeEnvironment (m, n, ENV_TEMP):
  # Initialize main matrix
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] is matStatus ; mat[i,j,1] is matTemp
  # Setting border status and environment temperature
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1:
        mat[i][j][0] = -1 # border
        mat[i][j][1] = ENV_TEMP # absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP
  # Initialize agents (viper and rodent)
  agents = []
  # Viper (row, column)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent (row, column, temperature)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  print(agents)
  # Setting agents in the matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # temp in rodent's position to its temp
  return mat, agents

def UpdateStatus (mat, agents):
  mat, agents = RodentWalk(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

def RodentWalk (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # if status in that cell is empty
      movementChoice.append([row+x, column+y])
  movementChoice.append([row, column]) # adding resting probability
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  if newPosition[0] == row and newPosition[1] == column:
    pass
  else:
    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

def ViperWalk (mat, agents):
  row, column = agents[0][0], agents[0][1]
  emptyNeighbors = []
  emptyNeighborsT = []
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # if status in that cell is empty or contains rodent
      emptyNeighbors.append([row+x, column+y])
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  maxT = max(emptyNeighborsT)

  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  
  newRow, newColumn = newPosition[0], newPosition[1]
  # Update matrix to viper's new position
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Update viper's information
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

def HeatDiffusion (m, n, mat, DIFF_RATE):
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE)* mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr:
    return True
  else:
    return False

# Main Program

m, n, ENV_TEMP, DIFF_RATE = 25, 25, 30, 0.1
matDisplay = []
phase = 0
while phase != 3:
  if phase == 0:
    mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
    phase += 1
  elif phase == 1:
    mat = HeatDiffusion(m, n, mat, DIFF_RATE)
    phase += 1

    matStatus =np.zeros((m+2,n+2))
    for i in range(m+2):
      for j in range(m+2):
        matStatus[i][j] = mat[i][j][1]
        if i  == agents[0][0] and j== agents[0][1]:     # viper
          matStatus[i][j] = 35
        elif i  == agents[1][0] and j== agents[1][1]:       # rodent
          matStatus[i][j] = 35


    matDisplay.append(matStatus)

  elif phase == 2:
    mat, agents = UpdateStatus(mat, agents)
    print(agents)
    if CheckPrey(agents[0], agents[1]): # Checking whether agents' position are the same (rodent is preyed)
      phase += 1
    else:
      phase = 1
# proceed to display all matrices in matDisplay
# proceed to display all matrices in matDisplay




# Menyiapkan figure, heatmap
fig = plt.figure()

# Fungsi init
def init():
  plt.clf()
  return None

# Fungsi iterasi animasi
def animate(i):
  plt.clf()
  ax = sns.heatmap(matDisplay[i-1],
                   cmap="magma",
                   square=True,
                   xticklabels=False,
                   yticklabels=False,
                   )

  return None

# Fungsi init
anim = animation.FuncAnimation(fig,
                               animate,
                               frames=range(1, len(matDisplay)+1, 1),
                               blit=False,
                               interval=250,
                               init_func=init)

# Kode tambahan jika dijalankan di colab/jupyter
from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')
anim


[[3, 5], [10, 16, 36.78009201051615]]
[[2, 6], [11, 15, 37.36231735137431]]
[[2, 5], [11, 14, 37.23835116838932]]
[[1, 6], [10, 15, 37.270996607036714]]
[[1, 7], [10, 16, 36.74623078656258]]
[[2, 7], [10, 15, 37.05522927305272]]
[[3, 6], [11, 14, 36.75113415228685]]
[[2, 5], [12, 14, 37.33367805530235]]
[[2, 4], [13, 13, 37.49018562134837]]
[[3, 5], [13, 14, 37.17865061616877]]
[[4, 6], [13, 14, 37.17865061616877]]
[[5, 7], [13, 13, 37.29897111323359]]
[[6, 8], [14, 13, 37.478869493709865]]
[[7, 9], [14, 13, 37.478869493709865]]
[[8, 10], [14, 12, 37.33340759793416]]
[[9, 11], [14, 12, 37.33340759793416]]
[[10, 12], [13, 11, 37.293309991769384]]
[[11, 13], [14, 10, 36.97811233719062]]
[[12, 12], [14, 11, 36.591992514920406]]
[[13, 11], [15, 10, 36.85075180926987]]
[[14, 11], [16, 9, 37.4912936048631]]
[[15, 10], [15, 10, 36.80701193067177]]


<Figure size 432x288 with 0 Axes>

# Program Kasus Ke-2 (Rodent dapat mendeteksi ular dan bergerak dua kali lebih cepet)
Program sama dengan kasus ke-1 dan hanya memiliki perubahan pada hal berikut.
* fungsi **UpdateStatus** agar memanggil fungsi bari **RodentWalk2**,
* perubahan nama fungsi **RodentWalk** ke **RodentWalk1**, dan
* penambahan fungsi baru, yaitu **RodentWalk2**.

Berikut asumsi-asumsi yang digunakan.
*   Absorbing border condition
*   Ketetanggaan Moore
* Ukuran matriks lingkungan 25x25
* Konstanta difusi 0.1
* Mangsa dapat bergerak dua kali lebih cepat dengan kemungkinan 90% ketika mendeteksi keberadaan *viper* di dekatnya.

In [ ]:
import numpy as np
def InitializeEnvironment (m, n, ENV_TEMP):
  # Initialize main matrix
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] is matStatus ; mat[i,j,1] is matTemp
  # Setting border status and environment temperature
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1:
        mat[i][j][0] = -1 # border
        mat[i][j][1] = ENV_TEMP # absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP
  # Initialize agents (viper and rodent)
  agents = []
  # Viper (row, column)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent (row, column, temperature)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  # Setting agents in the matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # temp in rodent's position to its temp
  return mat, agents

# Fungsi UpdateStatus hanya memiliki perubahan pada pemanggilan fungsi RodentWalk
def UpdateStatus (mat, agents):
  # Alih-alih memanggil RodentWalk seperti sebelumnya, akan dipanggil fungsi baru dari pergerakan rodent
  mat, agents = RodentWalk2(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

# Fungsi pergerakan rodent normal sebelumnya (RodentWalk) tetap ada dan hanya dilakukan perubahan nama jadi RodentWalk1
def RodentWalk1 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: 
      movementChoice.append([row+x, column+y])
  movementChoice.append([row, column]) 
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  if newPosition[0] == row and newPosition[1] == column:
    pass
  else:
    newRow, newColumn = newPosition[0], newPosition[1]
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

# Dibuat fungsi baru untuk algoritma pergerakan rodent untuk kasus kedua
def RodentWalk2 (mat, agents):
  # Pemerolehan informasi posisi rodent pada matriks lingkungan dan suhunya
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  # Akan dicek ada tidaknya viper di dekat rodent atau terdapat pada sel tetangganya
  isNearViper = False
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if isNearViper:
      break
    if mat[row+x][column+y][0] == 1: # Jika status pada sel tetangga adalah viper (kode 1),
      # terdapat peluang 90% bahwa rodent mewaspadai kehadiran viper dan akan bergerak dua kali lebih cepat
      if np.random.uniform() < 0.9:  
        isNearViper = True
  # Eksekusi pergerakan sesuai ada tidaknya viper
  if not isNearViper: # Jika tidak ada, dilakukan pergerakan normal sesuai kasus ke-1
    return RodentWalk1(mat, agents)
  else: # Jika ada, rodent akan bergerak lebih cepat
    rowv, colv = agents[0][0], agents[0][1] # Info posisi viper
    # Akan disimpan posisi sel yang kosong, jaraknya, dan pilihan-pilihan akhir yang paling memungkinkan
    emptyNeighbors, emptyNeighborsDist, movementChoice = [], [], []
    # Akan disimpan informasi sel tetangga kosong yang terletak dua kali lebih jauh dari rodent
    # yang merepresentasikan rodent bergerak dua kali lebih cepat dari normalnya
    for x,y in [[-2,2],[0,2],[2,2],[2,0],[2,-2],[0,-2],[-2,-2],[-2,0]]:
      if 0 <= row+x <= m+1 and 0 <= column+y <= n+1:
        if mat[row+x][column+y][0] == 0: 
          emptyNeighbors.append([row+x, column+y])
    # Akan dicari jarak sel-sel kosong tersebut terhadap posisi viper dan dicari jarak terjauhnya
    for x,y in emptyNeighbors:
      emptyNeighborsDist.append(abs(x-rowv)**2 + abs(y-colv)**2)
    maxDist = max(emptyNeighborsDist)
    # Sel dengan jarak terjauh akan dijadikan sel-sel pilihan akhir yang paling memungkinkan
    for i in range (len(emptyNeighbors)):
      if emptyNeighborsDist[i] == maxDist:
        movementChoice.append(emptyNeighbors[i])
    # Akan dipilih secara acak dari pilihan-pilihan sel posisi baru terbaik
    if len(movementChoice) == 1:
      newPosition = movementChoice[0]
    else:
      newPositionId = np.random.randint(0, len(movementChoice))
      newPosition = movementChoice[newPositionId]

    newRow, newColumn = newPosition[0], newPosition[1] # Informasi posisi baru
    # Matriks lingkungan akan di-update sesuai posisi baru rodent dan suhunya
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Informasi agen rodent juga di-update sesuai posisi baru dan ditentukan suhu acak barunya
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
    return mat, agents

def ViperWalk (mat, agents):
  row, column = agents[0][0], agents[0][1]
  emptyNeighbors = []
  emptyNeighborsT = []
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # if status in that cell is empty or contains rodent
      emptyNeighbors.append([row+x, column+y])
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  maxT = max(emptyNeighborsT)

  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  newRow, newColumn = newPosition[0], newPosition[1]
  # Update matrix to viper's new position
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Update viper's information
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

def HeatDiffusion (m, n, mat, DIFF_RATE):
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE) * mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr:
    return True
  else:
    return False

# Main Program
m, n, ENV_TEMP, DIFF_RATE = 25, 25, 20, 0.1
matDisplay = []
phase = 0
while phase != 3:
  if phase == 0:
    mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
    phase += 1
  elif phase == 1:
    mat = HeatDiffusion(m, n, mat, DIFF_RATE)
    phase += 1
    matDisplay.append(mat)
  elif phase == 2:
    print
    mat, agents = UpdateStatus(mat, agents)
    print(agents)
    if CheckPrey(agents[0], agents[1]): # Checking whether agents' position are the same (rodent is preyed)
      phase += 1
    else:
      phase = 1
# proceed to display all matrices in matDisplay

[[3, 20], [8, 4, 36.99119053619955]]
[[4, 20], [7, 3, 36.57965971009743]]
[[5, 19], [6, 2, 36.80162616689643]]
[[6, 18], [5, 1, 36.7829851500146]]
[[7, 17], [4, 1, 36.54281097328109]]
[[8, 16], [3, 2, 36.59617792158362]]
[[9, 15], [3, 2, 36.59617792158362]]
[[9, 14], [3, 2, 36.59617792158362]]
[[9, 13], [3, 2, 36.59617792158362]]
[[8, 12], [2, 1, 37.142305294461785]]
[[8, 11], [1, 2, 36.89014254529666]]
[[7, 10], [1, 1, 36.52701865411099]]
[[7, 9], [2, 2, 36.9649006060176]]
[[6, 8], [3, 3, 36.881322217389936]]
[[6, 7], [3, 2, 36.523744039028266]]
[[5, 6], [2, 3, 37.07304068616576]]
[[4, 5], [3, 3, 37.13798380726652]]
[[3, 4], [2, 2, 36.69108664686062]]
[[2, 3], [2, 1, 37.005166402879034]]
[[1, 2], [1, 2, 36.96514362192432]]


# Animasi Simulasi Kasus Ke-2
Program sebelumnya telah ditambahkan dengan tambahan sintaks untuk membuat animasi dari matriks-matriks lingkungan untuk tiap iterasi yang tersimpan pada *matDisplay*

In [ ]:
import numpy as np
def InitializeEnvironment (m, n, ENV_TEMP):
  # Initialize main matrix
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] is matStatus ; mat[i,j,1] is matTemp
  # Setting border status and environment temperature
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1:
        mat[i][j][0] = -1 # border
        mat[i][j][1] = ENV_TEMP # absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP
  # Initialize agents (viper and rodent)
  agents = []
  # Viper (row, column)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent (row, column, temperature)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  # Setting agents in the matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # temp in rodent's position to its temp
  return mat, agents

def UpdateStatus (mat, agents):
  mat, agents = RodentWalk2(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

def RodentWalk1 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # if status in that cell is empty
      movementChoice.append([row+x, column+y])
  movementChoice.append([row, column]) # adding resting probability
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  if newPosition[0] == row and newPosition[1] == column:
    pass
  else:
    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

def RodentWalk2 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2] # Rodent's position
  # Checking whether there's a snake nearby or not
  isNearViper = False
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if isNearViper:
      break
    if mat[row+x][column+y][0] == 1: # if status in that cell contains viper
      if np.random.uniform() < 0.9: # 90% chance that rodent will run twice as fast  
        isNearViper = True
  # Executing movement 
  if not isNearViper: # Normal walk
    return RodentWalk1(mat, agents)
  else: # Twice as fast as viper
    rowv, colv = agents[0][0], agents[0][1] # Viper's position
    emptyNeighbors, emptyNeighborsDist, movementChoice = [], [], []
    # Checking empty cells
    for x,y in [[-2,2],[0,2],[2,2],[2,0],[2,-2],[0,-2],[-2,-2],[-2,0]]:
      if 0 <= row+x <= m+1 and 0 <= column+y <= n+1:
        if mat[row+x][column+y][0] == 0: # if status in that cell is empty
          emptyNeighbors.append([row+x, column+y])
    # Choosing the furthest cells from viper
    for x,y in emptyNeighbors:
      emptyNeighborsDist.append(abs(x-rowv)**2 + abs(y-colv)**2)
    maxDist = max(emptyNeighborsDist)

    for i in range (len(emptyNeighbors)):
      if emptyNeighborsDist[i] == maxDist:
        movementChoice.append(emptyNeighbors[i])
  
    if len(movementChoice) == 1:
      newPosition = movementChoice[0]
    else:
      newPositionId = np.random.randint(0, len(movementChoice))
      newPosition = movementChoice[newPositionId]

    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
    return mat, agents

def ViperWalk (mat, agents):
  row, column = agents[0][0], agents[0][1]
  emptyNeighbors = []
  emptyNeighborsT = []
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # if status in that cell is empty or contains rodent
      emptyNeighbors.append([row+x, column+y])
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  maxT = max(emptyNeighborsT)

  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  
  newRow, newColumn = newPosition[0], newPosition[1]
  # Update matrix to viper's new position
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Update viper's information
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

def HeatDiffusion (m, n, mat, DIFF_RATE):
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE) * mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr:
    return True
  else:
    return False

# Main Program

m, n, ENV_TEMP, DIFF_RATE = 25, 25, 30, 0.1
matDisplay = []
phase = 0
while phase != 3:
  if phase == 0:
    mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
    phase += 1
  elif phase == 1:
    mat = HeatDiffusion(m, n, mat, DIFF_RATE)
    phase += 1

    matStatus =np.zeros((m+2,n+2))
    for i in range(m+2):
      for j in range(m+2):
        matStatus[i][j] = mat[i][j][1]
        if i  == agents[0][0] and j== agents[0][1]:     # viper
          matStatus[i][j] = 35
        elif i  == agents[1][0] and j== agents[1][1]:       # rodent
          matStatus[i][j] = 35


    matDisplay.append(matStatus)

  elif phase == 2:
    mat, agents = UpdateStatus(mat, agents)
    print(agents)
    if CheckPrey(agents[0], agents[1]): # Checking whether agents' position are the same (rodent is preyed)
      phase += 1
    else:
      phase = 1
# proceed to display all matrices in matDisplay




# Menyiapkan figure, heatmap
fig = plt.figure()

# Fungsi init
def init():
  plt.clf()
  return None

# Fungsi iterasi animasi
def animate(i):
  plt.clf()
  ax = sns.heatmap(matDisplay[i-1],
                   cmap="magma",
                   square=True,
                   xticklabels=False,
                   yticklabels=False,
                   )

  return None

# Fungsi init
anim = animation.FuncAnimation(fig,
                               animate,
                               frames=range(1, len(matDisplay)+1, 1),
                               blit=False,
                               interval=250,
                               init_func=init)

# Kode tambahan jika dijalankan di colab/jupyter
from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')
anim

[[11, 24], [20, 18, 36.784943504123056]]
[[10, 24], [20, 17, 36.5097299128163]]
[[10, 23], [20, 17, 36.5097299128163]]
[[11, 23], [21, 16, 36.89706520108265]]
[[12, 24], [21, 16, 36.89706520108265]]
[[12, 25], [22, 15, 37.122422658198644]]
[[11, 24], [23, 16, 36.525160169994905]]
[[10, 25], [23, 17, 36.882896212905784]]
[[9, 24], [24, 17, 37.019014504723934]]
[[8, 25], [25, 16, 36.96859887251866]]
[[7, 25], [24, 16, 37.284622001665696]]
[[8, 24], [24, 17, 37.278904956458774]]
[[9, 23], [24, 17, 37.278904956458774]]
[[10, 22], [24, 18, 36.88882357935965]]
[[11, 21], [24, 19, 36.76474572255624]]
[[12, 20], [25, 18, 36.83873580925035]]
[[13, 19], [24, 19, 36.718478497813805]]
[[14, 18], [23, 20, 36.94602267505447]]
[[15, 17], [22, 20, 37.47853167299965]]
[[16, 17], [21, 19, 36.60412476344451]]
[[17, 17], [20, 18, 37.07334668400827]]
[[18, 17], [21, 19, 37.48692312483105]]
[[19, 18], [22, 18, 36.881384605636946]]
[[20, 19], [22, 18, 36.881384605636946]]
[[21, 18], [23, 17, 36.8747448204984

<Figure size 432x288 with 0 Axes>

# Simulasi untuk Mencari Rata-Rata Iterasi Kasus Ke-1
Berdasarkan program kasus ke-1, telah ditambahkan modifikasi agar simulasi dapat dijalankan sebanyak 100 kali sehingga rata-rata iterasi yang dibutuhkan untuk *viper* memangsa dapat ditentukan.

In [ ]:
import numpy as np
def InitializeEnvironment (m, n, ENV_TEMP):
  # Initialize main matrix
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] is matStatus ; mat[i,j,1] is matTemp
  # Setting border status and environment temperature
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1:
        mat[i][j][0] = -1 # border
        mat[i][j][1] = ENV_TEMP # absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP
  # Initialize agents (viper and rodent)
  agents = []
  # Viper (row, column)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent (row, column, temperature)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  # Setting agents in the matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # temp in rodent's position to its temp
  return mat, agents

def UpdateStatus (mat, agents):
  mat, agents = RodentWalk1(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

def RodentWalk1 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # if status in that cell is empty
      movementChoice.append([row+x, column+y])
  movementChoice.append([row, column]) # adding resting probability
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  if newPosition[0] == row and newPosition[1] == column:
    pass
  else:
    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

def RodentWalk2 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2] # Rodent's position
  # Checking whether there's a snake nearby or not
  isNearViper = False
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if isNearViper:
      break
    if mat[row+x][column+y][0] == 1: # if status in that cell contains viper
      if np.random.uniform() < 0.9: # 90% chance that rodent will run twice as fast  
        isNearViper = True
  # Executing movement 
  if not isNearViper: # Normal walk
    return RodentWalk1(m, n, mat, agents)
  else: # Twice as fast as viper
    rowv, colv = agents[0][0], agents[0][1] # Viper's position
    emptyNeighbors, emptyNeighborsDist, movementChoice = [], [], []
    # Checking empty cells
    for x,y in [[-2,2],[0,2],[2,2],[2,0],[2,-2],[0,-2],[-2,-2],[-2,0]]:
      if 0 <= row+x <= m+1 and 0 <= column+y <= n+1:
        if mat[row+x][column+y][0] == 0: # if status in that cell is empty
          emptyNeighbors.append([row+x, column+y])
    # Choosing the furthest cells from viper
    for x,y in emptyNeighbors:
      emptyNeighborsDist.append(abs(x-rowv)**2 + abs(y-colv)**2)
    maxDist = max(emptyNeighborsDist)

    for i in range (len(emptyNeighbors)):
      if emptyNeighborsDist[i] == maxDist:
        movementChoice.append(emptyNeighbors[i])
  
    if len(movementChoice) == 1:
      newPosition = movementChoice[0]
    else:
      newPositionId = np.random.randint(0, len(movementChoice))
      newPosition = movementChoice[newPositionId]

    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
    return mat, agents

def ViperWalk (mat, agents):
  row, column = agents[0][0], agents[0][1]
  emptyNeighbors = []
  emptyNeighborsT = []
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # if status in that cell is empty or contains rodent
      emptyNeighbors.append([row+x, column+y])
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  maxT = max(emptyNeighborsT)

  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  
  newRow, newColumn = newPosition[0], newPosition[1]
  # Update matrix to viper's new position
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Update viper's information
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

def HeatDiffusion (m, n, mat, DIFF_RATE):
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE) * mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr:
    return True
  else:
    return False

# Main Program
matLength = []
for k in range(100):
  m, n, ENV_TEMP, DIFF_RATE = 25, 25, 25, 0.1
  matDisplay = []
  phase = 0
  while phase != 3:
    if phase == 0:
      mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
      phase += 1
    elif phase == 1:
      mat = HeatDiffusion(m, n, mat, DIFF_RATE)
      phase += 1
      matDisplay.append(mat)
    elif phase == 2:
      print
      mat, agents = UpdateStatus(mat, agents)
      if CheckPrey(agents[0], agents[1]): # Checking whether agents' position are the same (rodent is preyed)
        phase += 1
      else:
        phase = 1
  matLength.append(len(matDisplay))

average = sum(matLength)/len(matLength)
print("Dibutuhkan rata-rata", average, "iterasi")


Dibutuhkan rata-rata 15.65 iterasi


# Simulasi untuk Mencari Rata-Rata Iterasi Kasus Ke-2
Berdasarkan program kasus ke-1, telah ditambahkan modifikasi agar simulasi dapat dijalankan sebanyak 100 kali sehingga rata-rata iterasi yang dibutuhkan untuk *viper* memangsa dapat ditentukan.

In [ ]:
import numpy as np
def InitializeEnvironment (m, n, ENV_TEMP):
  # Initialize main matrix
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] is matStatus ; mat[i,j,1] is matTemp
  # Setting border status and environment temperature
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1:
        mat[i][j][0] = -1 # border
        mat[i][j][1] = ENV_TEMP # absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP
  # Initialize agents (viper and rodent)
  agents = []
  # Viper (row, column)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent (row, column, temperature)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  # Setting agents in the matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # temp in rodent's position to its temp
  return mat, agents

def UpdateStatus (mat, agents):
  mat, agents = RodentWalk2(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

def RodentWalk1 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # if status in that cell is empty
      movementChoice.append([row+x, column+y])
  movementChoice.append([row, column]) # adding resting probability
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  if newPosition[0] == row and newPosition[1] == column:
    pass
  else:
    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

def RodentWalk2 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2] # Rodent's position
  # Checking whether there's a snake nearby or not
  isNearViper = False
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if isNearViper:
      break
    if mat[row+x][column+y][0] == 1: # if status in that cell contains viper
      if np.random.uniform() < 0.9: # 90% chance that rodent will run twice as fast  
        isNearViper = True
  # Executing movement 
  if not isNearViper: # Normal walk
    return RodentWalk1(mat, agents)
  else: # Twice as fast as viper
    rowv, colv = agents[0][0], agents[0][1] # Viper's position
    emptyNeighbors, emptyNeighborsDist, movementChoice = [], [], []
    # Checking empty cells
    for x,y in [[-2,2],[0,2],[2,2],[2,0],[2,-2],[0,-2],[-2,-2],[-2,0]]:
      if 0 <= row+x <= m+1 and 0 <= column+y <= n+1:
        if mat[row+x][column+y][0] == 0: # if status in that cell is empty
          emptyNeighbors.append([row+x, column+y])
    # Choosing the furthest cells from viper
    for x,y in emptyNeighbors:
      emptyNeighborsDist.append(abs(x-rowv)**2 + abs(y-colv)**2)
    maxDist = max(emptyNeighborsDist)

    for i in range (len(emptyNeighbors)):
      if emptyNeighborsDist[i] == maxDist:
        movementChoice.append(emptyNeighbors[i])
  
    if len(movementChoice) == 1:
      newPosition = movementChoice[0]
    else:
      newPositionId = np.random.randint(0, len(movementChoice))
      newPosition = movementChoice[newPositionId]

    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
    return mat, agents

def ViperWalk (mat, agents):
  row, column = agents[0][0], agents[0][1]
  emptyNeighbors = []
  emptyNeighborsT = []
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # if status in that cell is empty or contains rodent
      emptyNeighbors.append([row+x, column+y])
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  maxT = max(emptyNeighborsT)

  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  
  newRow, newColumn = newPosition[0], newPosition[1]
  # Update matrix to viper's new position
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Update viper's information
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

def HeatDiffusion (m, n, mat, DIFF_RATE):
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE) * mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr:
    return True
  else:
    return False

# Main Program
matLength = []
for k in range(100):
  m, n, ENV_TEMP, DIFF_RATE = 25, 25, 30, 0.1
  matDisplay = []
  phase = 0
  while phase != 3:
    if phase == 0:
      mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
      phase += 1
    elif phase == 1:
      mat = HeatDiffusion(m, n, mat, DIFF_RATE)
      phase += 1
      matDisplay.append(mat)
    elif phase == 2:
      print
      mat, agents = UpdateStatus(mat, agents)
      if CheckPrey(agents[0], agents[1]): # Checking whether agents' position are the same (rodent is preyed)
        phase += 1
      else:
        phase = 1
  matLength.append(len(matDisplay))

average = sum(matLength)/len(matLength)
print("Dibutuhkan rata-rata", average, "iterasi")


Dibutuhkan rata-rata 23.83 iterasi
